# Installations

In [ ]:
!pip install dlib numpy pandas matplotlib opencv-python imutils

# Importing

In [2]:
from imutils import paths, face_utils
import numpy as np
import cv2
import matplotlib.pyplot as plt
import dlib
import pandas as pd

# Dlib face detector and shape predictor

In [3]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('model.dat')

In [13]:
# dataset paths
dataset_path = 'datasets/'
open_eye_path = dataset_path + 'open/'
closed_eye_path = dataset_path + 'close/'

In [15]:
IMG_SIZE = (34, 26)

In [28]:
imageList = paths.list_images(dataset_path)

In [26]:
# Eye crop from image
def crop_eye(img, eye_points):
    x1, y1 = np.amin(eye_points, axis=0)
    x2, y2 = np.amax(eye_points, axis=0)
    cx, cy = (x1 + x2) / 2, (y1 + y2) / 2

    w = (x2 - x1) * 1.2
    h = w * IMG_SIZE[1] / IMG_SIZE[0]

    margin_x, margin_y = w / 2, h / 2

    min_x, min_y = int(cx - margin_x), int(cy - margin_y)
    max_x, max_y = int(cx + margin_x), int(cy + margin_y)

    eye_rect = np.rint([min_x, min_y, max_x, max_y]).astype(np.int)

    eye_img = img[eye_rect[1]: eye_rect[3], eye_rect[0]:eye_rect[2]]

    return eye_img, eye_rect

In [29]:
# Eye detection
for image in imageList:
    # read image
    img = cv2.imread(image)

    # convert to gray
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # detect faces
    faces = detector(gray)

    for face in faces:
        # get landmarks
        landmarks = predictor(gray, face)
        shapes = face_utils.shape_to_np(landmarks)

        left_eye_img, left_eye_rect = crop_eye(gray, shapes[36:42])
        right_eye_img, right_eye_rect = crop_eye(gray, shapes[42:48])

        right_eye_img = cv2.flip(right_eye_img, flipCode=1)

        if 'open' in image:
            cv2.imwrite(dataset_path + 'crop_open/' + 'left_eye' + image.split('/')[-1], left_eye_img)
            cv2.imwrite(dataset_path + 'crop_open/' + 'right_eye' + image.split('/')[-1], right_eye_img)
        elif 'close' in image:
            cv2.imwrite(dataset_path + 'crop_close/' + 'left_eye' + image.split('/')[-1], left_eye_img)
            cv2.imwrite(dataset_path + 'crop_close/' + 'right_eye' + image.split('/')[-1], right_eye_img)


/var/folders/1t/88mj6ftx7kzcj3p0wtqxbcl40000gn/T/ipykernel_3132/1968544041.py:14: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eye_rect = np.rint([min_x, min_y, max_x, max_y]).astype(np.int)


In [57]:
# eye image lists
open_eyes_list = paths.list_images("/Users/yoshlikmedia/Projects/blink-detection/models/datasets/crop_close")
close_eyes_list = paths.list_images("/Users/yoshlikmedia/Projects/blink-detection/models/datasets/crop_open")

In [58]:
# all images 
all_images = list(open_eyes_list) + list(close_eyes_list)

In [45]:
# shufler all images
from random import shuffle
shuffle(all_images)

# Add Dataframe

In [60]:
df = pd.DataFrame()

for image in all_images:
    img = cv2.imread(image) # RGB 3D
    img = cv2.resize(img, dsize=IMG_SIZE) 
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # RGB to GRAY 2D

    img_list = []

    for row in img:
        for pixel in row:
            img_list.append(pixel)

    image_arr = str(img_list)
    if 'open' in image:
        label = 'open'
    elif 'close' in image:
        label = 'close'

    df = df.append({'state': label, 'image': image_arr}, ignore_index=True)
df.to_csv('dataset.csv', index=False)


/var/folders/1t/88mj6ftx7kzcj3p0wtqxbcl40000gn/T/ipykernel_3132/2595441371.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'state': label, 'image': image_arr}, ignore_index=True)
/var/folders/1t/88mj6ftx7kzcj3p0wtqxbcl40000gn/T/ipykernel_3132/2595441371.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'state': label, 'image': image_arr}, ignore_index=True)
/var/folders/1t/88mj6ftx7kzcj3p0wtqxbcl40000gn/T/ipykernel_3132/2595441371.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'state': label, 'image': image_arr}, ignore_index=True)
/var/folders/1t/88mj6ftx7kzcj3p0wtqxbcl40000gn/T/ipykernel_3132/2595441371.py:20: FutureWarning: The frame.append method is deprecate

In [61]:
df.head(20)

,state,image
0,close,"[104, 107, 109, 111, 113, 115, 123, 131, 132, ..."
1,close,"[106, 105, 108, 104, 103, 103, 104, 104, 104, ..."
2,close,"[38, 39, 43, 44, 45, 47, 49, 47, 46, 47, 47, 4..."
3,close,"[82, 82, 82, 81, 81, 81, 85, 84, 87, 89, 91, 9..."
4,close,"[108, 104, 96, 107, 108, 106, 104, 107, 105, 1..."
5,close,"[119, 116, 119, 125, 131, 124, 126, 128, 129, ..."
6,close,"[115, 116, 119, 122, 125, 126, 123, 123, 119, ..."
7,close,"[113, 115, 115, 114, 114, 111, 111, 111, 109, ..."
8,close,"[85, 84, 84, 87, 91, 93, 94, 97, 97, 96, 94, 9..."
9,close,"[116, 110, 114, 118, 115, 115, 113, 109, 105, ..."
